In [ ]:
import subprocess
from codecarbon import track_emissions

# Build C/C++ and Fortran executables
c_compiler = "gcc"
f_compiler = "gfortran"

print("C compiler details: \n")
command = c_compiler + " --version"
subprocess.run([command], shell=True)

print("\n")

print("Fortran compiler details: \n")
command = f_compiler + " --version"
subprocess.run([command], shell=True)

command = c_compiler + " ./code/nqueens.c -o nqueens_c.exe"
subprocess.run([command], shell=True)

command = f_compiler + " ./code/nqueens.f90 -o nqueens_f.exe"
subprocess.run([command], shell=True)            


#
# Set up carbon tracking demo functions. Note the code is run via the subprocess module, and as such as there
# is a computational overhead that will significantly skew results for short run times. You can ameliorate this,
# by running the same algoritm multiple times. Finally BEWARE the demo code provided as is writes to stdout, 
# and this can significantly skew the results if you are just interested "algorithm execute time". 
# 

project_name = "nqueens"
measure_power_secs = 5
country_iso_code = "GBR"
offline = True

@track_emissions(
    project_name = project_name,
    experiment_id="python_demo",
    output_file="python_demo.csv",
    measure_power_secs=measure_power_secs,
    country_iso_code=country_iso_code,
    offline=offline,
)
def python_demo_n_queens(size, repeats):
    command = "python ./code/nqueens.py --size {} --repeats {}".format(size, repeats) 
    subprocess.run([command], shell=True, capture_output=True, text=True)

@track_emissions(
    project_name = project_name,
    experiment_id="c_demo",
    output_file="c_demo.csv",
    measure_power_secs=measure_power_secs,
    country_iso_code=country_iso_code,
    offline=offline,
)
def c_demo_n_queens(size, repeats):
    command = "./nqueens_c.exe {} {}".format(size, repeats) 
    subprocess.run([command], shell=True, capture_output=True, text=True)

@track_emissions(
    project_name = project_name,
    experiment_id="fortran_demo",
    output_file="fortran_demo.csv",
    measure_power_secs=measure_power_secs,
    country_iso_code=country_iso_code,
    offline=offline,
)
def fortran_demo_n_queens(size, repeats):
    command = "./nqueens_f.exe {} {}".format(size, repeats) 
    subprocess.run([command], shell=True, capture_output=True, text=True)

In [ ]:
# Set up experimental run parameters. If you run this notebook for first time, you might want to
# modify the list and only run for small N.
experiments = [
    (12, 100),
    (13, 30),
    (14, 10),
    (15, 1),
]

In [ ]:
for experiment in experiments:
    python_demo_n_queens(size=experiment[0], repeats=experiment[1])

In [ ]:
for experiment in experiments:
    fortran_demo_n_queens(size=experiment[0], repeats=experiment[1])

In [ ]:
for experiment in experiments:
    c_demo_n_queens(size=experiment[0], repeats=experiment[1])

In [ ]:
import pandas as pd

In [ ]:
df_python = pd.read_csv("python_demo.csv")
df_fortran = pd.read_csv("fortran_demo.csv")
df_c = pd.read_csv("c_demo.csv")

In [ ]:
# Set some colours
darkgreen = "#003c3c"
deepgreen = "#007d69"
highlightgreen = "#00dca5"
woodbrown = "#93272c" 
treebrown = "#b46a55" 
rosered = "#e60000"
warmred = "#f9423a"
purepurple = "#702082"
lowpurple = "#9569be"
coralorange = "#fc4c02"
peach = "#ff7f41" 
hay = "#ffc72c"
sunyellow = "#f3d54e" 
deepsea = "#250e62"
morningsky = "#69b3e7"
sunsetpink = "#a68699" 
lightpink = "#f4c3cc" 
heavyblack = "#000000" 
stonegrey = "#888b8d"

colours_contrast = [
    darkgreen,
    highlightgreen,
    coralorange,
    purepurple,
    treebrown,
    deepgreen,
    peach,
    morningsky,
    hay,
    rosered,
    sunyellow,
    deepsea,
    stonegrey,
    sunsetpink,
    lightpink,
    warmred,
    lowpurple,
    woodbrown,
    heavyblack,
]

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

# set width of bars
barWidth = 0.25
n_experiments = 4

# Set position of bars on horisontal axis
br1 = np.arange(n_experiments) 
br2 = [x + barWidth for x in br1] 
br3 = [x + barWidth for x in br2]

# Set bar group labels
p_energy = df_python["energy_consumed"].to_numpy() * 1000
f_energy = df_fortran["energy_consumed"].to_numpy() * 1000
c_energy = df_c["energy_consumed"].to_numpy() * 1000
experiment_labels = [
    "N = 12\n runs = 100\n" + "$E_{max}$ = " + str(round(max(p_energy[0], f_energy[0], c_energy[0]), 2)) + " Wh",
    "N = 13\n runs = 30\n" + "$E_{max}$ = " + str(round(max(p_energy[1], f_energy[1], c_energy[1]), 2)) + " Wh",
    "N = 14\n runs = 10\n" + "$E_{max}$ = " + str(round(max(p_energy[2], f_energy[2], c_energy[2]), 2)) + " Wh",
    "N = 15\n runs = 1\n" + "$E_{max}$ = " + str(round(max(p_energy[3], f_energy[3], c_energy[3]), 2)) + " Wh",
]

#
# Make total emissions plot
#
# set height of bars
p_emissions = df_python["emissions"].to_numpy() * 1000
f_emissions = df_fortran["emissions"].to_numpy() * 1000
c_emissions = df_c["emissions"].to_numpy() * 1000

# colour_index
col_idx = 6

# Create the plot
fig = plt.figure(figsize = (16, 8))
ax = fig.add_subplot()
plt.bar(br1, p_emissions, color = colours_contrast[col_idx], width = barWidth, label = 'Python') 
plt.bar(br2, f_emissions, color = colours_contrast[col_idx + 1], width = barWidth, label = 'Fortran') 
plt.bar(br3, c_emissions, color = colours_contrast[col_idx + 2], width = barWidth, label = 'C/C++') 

# Hide the right and top spines
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)

# Plot labels
plt.title('Total CO2eq emissions in grams', fontsize = 28) 
plt.xticks([r + barWidth for r in range(len(experiment_labels))], experiment_labels)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tick_params(left = False)
plt.tick_params(bottom = False)

plt.legend(fontsize=18)
plt.savefig("emissions.png")
#plt.show() 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

# set width of bars
barWidth = 0.25
n_experiments = 4

# Set position of bars on horisontal axis
br1 = np.arange(n_experiments) 
br2 = [x + barWidth for x in br1] 
br3 = [x + barWidth for x in br2]

# Set bar group labels
p_energy = df_python["energy_consumed"].to_numpy() * 1000
f_energy = df_fortran["energy_consumed"].to_numpy() * 1000
c_energy = df_c["energy_consumed"].to_numpy() * 1000
experiment_labels = [
    "N = 12\n runs = 100\n" + "$E_{max}$ = " + str(round(max(p_energy[0], f_energy[0], c_energy[0]), 2)) + " Wh",
    "N = 13\n runs = 30\n" + "$E_{max}$ = " + str(round(max(p_energy[1], f_energy[1], c_energy[1]), 2)) + " Wh",
    "N = 14\n runs = 10\n" + "$E_{max}$ = " + str(round(max(p_energy[2], f_energy[2], c_energy[2]), 2)) + " Wh",
    "N = 15\n runs = 1\n" + "$E_{max}$ = " + str(round(max(p_energy[3], f_energy[3], c_energy[3]), 2)) + " Wh",
]

#
# Make total ratio of RAM to CPU energy usage plot
#
# set height of bars
p_emissions = df_python["cpu_energy"].to_numpy() / df_python["ram_energy"].to_numpy()
f_emissions = df_fortran["cpu_energy"].to_numpy() / df_fortran["ram_energy"].to_numpy()
c_emissions = df_c["cpu_energy"].to_numpy() / df_c["ram_energy"].to_numpy()

# colour_index
col_idx = 6

# Create the plot
fig = plt.figure(figsize = (16, 8))
ax = fig.add_subplot()
plt.bar(br1, p_emissions, color = colours_contrast[col_idx], width = barWidth, label = 'Python') 
plt.bar(br2, f_emissions, color = colours_contrast[col_idx + 1], width = barWidth, label = 'Fortran') 
plt.bar(br3, c_emissions, color = colours_contrast[col_idx + 2], width = barWidth, label = 'C/C++') 

# Hide the right and top spines
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)

# set vertical axis limits
ax.set_ylim([0, 3])

# Plot labels
plt.title('Ratio of RAM to CPU energy usage', fontsize = 28)
plt.xticks([r + barWidth for r in range(len(experiment_labels))], experiment_labels)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tick_params(left = False)
plt.tick_params(bottom = False)

plt.legend(fontsize=18)
plt.savefig("cpu_to_ram.png")
#plt.show() 